In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Dateien einlesen
idn_df = pd.read_excel("idn.xlsx", dtype=str)  # IDNs laden
keywords_df = pd.read_excel("Schlagwort.xlsx", dtype=str)  # Schlagwörter mit Spalte "Alleinstehend" laden

# Schlagwörter in eine Liste mit Status "Alleinstehend" umwandeln
keywords = keywords_df.iloc[:, 0].dropna().tolist()
alone_status = keywords_df.set_index(keywords_df.columns[0])["Alleinstehend"].fillna("").to_dict()  # "x" oder leer

# Funktion zum Abrufen und Durchsuchen der Texte
def find_keywords(idn):
    url_text = f"https://d-nb.info/{idn}/04/text"
    url_pdf = f"https://d-nb.info/{idn}/04/pdf"
    try:
        response = requests.get(url_text, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text().lower()  # Ganze Seite als Text
            
            word_counts = {}  # Speichert Schlagwort und Anzahl
            
            for word in keywords:
                # Suche nach dem Schlagwort als Teil eines Wortes (z.B. "museum" in "Volksmuseum")
                pattern = re.compile(rf"\b\w*{re.escape(word.lower())}\w*\b", re.IGNORECASE)
                matches = pattern.findall(text)  # Finde alle Übereinstimmungen
                count = len(matches)  # Zähle die Vorkommen
                
                if count > 0:
                    word_counts[word] = count  # Speichert Wort + Anzahl
            
            # Prüfen, ob der Eintrag gespeichert werden soll
            if len(word_counts) >= 2:  
                # Mindestens 2 verschiedene Schlagwörter -> Speichern
                formatted_words = [f"{word} ({count})" for word, count in word_counts.items()]
                return url_text, url_pdf, ", ".join(formatted_words)
            elif len(word_counts) == 1:  
                # Falls nur ein einziges Schlagwort gefunden wurde
                single_word = list(word_counts.keys())[0]
                count = word_counts[single_word]
                
                # Nur speichern, wenn es mindestens 2-mal vorkommt und ein "x" in "Alleinstehend" hat
                if count >= 2 and alone_status.get(single_word, "").strip().lower() == "x":
                    return url_text, url_pdf, f"{single_word} ({count})"

    except requests.RequestException:
        return url_text, url_pdf, "Fehler"
    
    return url_text, url_pdf, None  # Falls keine Bedingung erfüllt ist, Rückgabe als None

# Neue Spalten füllen
idn_df[["URL", "PDF-URL", "Gefundenes Schlagwort"]] = idn_df.iloc[:, 0].apply(find_keywords).apply(pd.Series)

# Nur Zeilen mit gefundenen Schlagwörtern behalten
idn_df = idn_df.dropna(subset=["Gefundenes Schlagwort"])

# Spaltennamen setzen
idn_df.rename(columns={idn_df.columns[0]: "IDN"}, inplace=True)

# Ergebnis speichern, nur wenn es Treffer gibt
if not idn_df.empty:
    idn_df.to_excel("idn_mit_schlagwort.xlsx", index=False)
    print("Die Datei 'idn_mit_schlagwort.xlsx' wurde erfolgreich erstellt!")
else:
    print("Keine passenden Schlagwörter gefunden – es wurde keine Datei erstellt.")



C:\Users\sickerti\AppData\Local\Temp\ipykernel_10496\3215325607.py:21: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, "html.parser")


Die Datei 'idn_mit_schlagwort.xlsx' wurde erfolgreich erstellt!
